In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np


In [2]:
#loading file
amino_acid_df = pd.read_excel("data/AminoAcid.xlsx", header=None)
amino_acid_df.columns = ['protein', 'sequence']

c:\Users\Jean-Luc\.conda\envs\protein-protein\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
amino_acid_df.head()

,protein,sequence
0,1EP9_1,VQLKGRDLLTLKNFTGEEIKYMLWLSADLKFRIKQKGEYLPLLQGK...
1,1BH9_1,LFSKELRCMMYGFGDDQNPYTESVDILEDLVIEFITEMTHKAMSI
2,1G96_1,VGGPMDASVEEEGVRRALDFAVGEYNKASNDMYHSRALQVVRARKQ...
3,1CYV_1,MIPGGLSEAKPATPEIQEIVDKVKPQLEEKTNETYGKLEAVQYKTQ...
4,1KE5_1,MENFQKVEKIGEGTYGVVYKARNKLTGEVVALKKIR


In [4]:
#spliting the sequences in letters
amino_acid_df['sequence'] = amino_acid_df['sequence'].apply(lambda seq: list(seq))

In [5]:
amino_acid_df.head()

,protein,sequence
0,1EP9_1,"[V, Q, L, K, G, R, D, L, L, T, L, K, N, F, T, ..."
1,1BH9_1,"[L, F, S, K, E, L, R, C, M, M, Y, G, F, G, D, ..."
2,1G96_1,"[V, G, G, P, M, D, A, S, V, E, E, E, G, V, R, ..."
3,1CYV_1,"[M, I, P, G, G, L, S, E, A, K, P, A, T, P, E, ..."
4,1KE5_1,"[M, E, N, F, Q, K, V, E, K, I, G, E, G, T, Y, ..."


In [27]:
unique_amino_acid = list(set([item for sublist in amino_acid_df['sequence'] for item in sublist]))
unique_amino_acid.sort()

In [11]:
max_lenght_amino_acid = len(max(amino_acid_df['sequence'], key=len))

In [28]:
# Function to one-hot encode a sequence of amino acid.
# The output is a matrix of max_lenght_amino_acid (1965) x unique_amino_acid (21). Sequence shorter than max_lenght_amino_acid are filled with 0. 
def one_hot_encode(seq):
    matrix = np.zeros((max_lenght_amino_acid, len(unique_amino_acid)))
    for idx, elem in enumerate(seq):
        matrix[idx][unique_amino_acid.index(elem)] = 1
    return matrix


In [21]:
amino_acid_df['seq_one_hot'] = amino_acid_df['sequence'].apply(lambda seq: one_hot_encode(seq))
